In [1]:
import os
from keras.preprocessing import image
import cv2

import random
import numpy as np

In [2]:
#creating two categories
categories=['nomask','mask']

In [3]:
data=[]

for category in categories:
    path=os.path.join('dataset',category)
    label=categories.index(category)
    for file in os.listdir(path):
        img_path=os.path.join(path,file)
        img = cv2.imread(img_path)
        img=cv2.resize(img,(224,224))
        data.append([img,label])

In [4]:
random.shuffle(data)

In [5]:
x=[]
y=[]
for features,label in data:
    x.append(features)
    y.append(label)

In [6]:
x=np.array(x)
y=np.array(y)

In [7]:
#standaising x
x=x/255

In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.2)

In [9]:
from keras.applications.vgg16 import VGG16
vgg=VGG16()

In [10]:
from keras import Sequential
model=Sequential()

In [11]:
#to remove last prediction layer of VGG16
for layer in vgg.layers[:-1]:
    model.add(layer)

In [12]:
#to freeze layer for training
for layer in model.layers:
    layer.trainable=False
    

In [13]:
#to add new layer
from keras.layers import Dense

In [14]:
model.add(Dense(1,activation='sigmoid'))

In [15]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [16]:
model.fit(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Epoch 1/5
102/102 [==============================] - 850s 8s/step - loss: 0.4558 - accuracy: 0.8251 - val_loss: 0.3295 - val_accuracy: 0.8916
Epoch 2/5
102/102 [==============================] - 766s 8s/step - loss: 0.2782 - accuracy: 0.9129 - val_loss: 0.2524 - val_accuracy: 0.9224
Epoch 3/5
102/102 [==============================] - 767s 8s/step - loss: 0.2241 - accuracy: 0.9276 - val_loss: 0.2035 - val_accuracy: 0.9372
Epoch 4/5
102/102 [==============================] - 757s 7s/step - loss: 0.1968 - accuracy: 0.9360 - val_loss: 0.1805 - val_accuracy: 0.9446
Epoch 5/5
102/102 [==============================] - 805s 8s/step - loss: 0.1863 - accuracy: 0.9409 - val_loss: 0.1687 - val_accuracy: 0.9446


In [17]:
cap=cv2.VideoCapture(0)

In [23]:
def detect_face_mask(img):
    y_pred=model.predict(img.reshape(1,224,224,3))
    return y_pred[0][0]

In [19]:
def draw_label(img,text,pos,bg_color):
    
    text_size=cv2.getTextSize(text,cv2.FONT_HERSHEY_SIMPLEX,1,cv2.FILLED)
    end_x=pos[0]+text_size[0][0]+2
    end_y=pos[1]+text_size[0][1]-2
    
    cv2.rectangle(img,pos,(end_x,end_y),bg_color,cv2.FILLED)
    cv2.putText(img,text,pos,cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),1,cv2.LINE_AA)

In [27]:
while(True):
    ret,frame=cap.read()
    
    #call detection method
    img=cv2.resize(frame,(224,224))
    y_pred=detect_face_mask(img)
    print(y_pred)
    if y_pred<=0.5:
        draw_label(frame,'nomask',(30,30),(0,0,255))
    else:
        draw_label(frame,'mask',(30,30),(0,255,0))
        
        
    cv2.imshow("window",frame)
    if(cv2.waitKey(1) & 0xff==ord('x')):
        break

cv2.destroyAllWindows()

1/1 [==============================] - 0s 479ms/step
0.06900032
1/1 [==============================] - 0s 494ms/step
0.11963566
1/1 [==============================] - 1s 500ms/step
0.19795258
1/1 [==============================] - 0s 480ms/step
0.14959787
1/1 [==============================] - 1s 511ms/step
0.03331797
1/1 [==============================] - 0s 493ms/step
0.08340014
1/1 [==============================] - 1s 503ms/step
0.11601743
1/1 [==============================] - 0s 496ms/step
0.12548505
1/1 [==============================] - 0s 488ms/step
0.20545751
1/1 [==============================] - 0s 494ms/step
0.12730205
1/1 [==============================] - 0s 483ms/step
0.21966921
1/1 [==============================] - 0s 481ms/step
0.65532464
1/1 [==============================] - 0s 489ms/step
0.27181566
1/1 [==============================] - 0s 487ms/step
0.8623757
1/1 [==============================] - 0s 481ms/step
0.06787033
1/1 [==============================] - 0s

KeyboardInterrupt: 